<a href="https://colab.research.google.com/github/erha500/IMLO-Open-Assessment/blob/main/IMLO_Open_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision.transforms import v2

In [20]:
train_transform = v2.Compose([
    v2.RandomResizedCrop([224,224], scale=[0.25,1.0], ratio=[1.0,1.0]),
    v2.RandomPerspective(),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(90),
    #v2.RandomVerticalFlip(0.5)
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = v2.Compose([
  v2.Resize([224,224]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

C:\Users\erhan\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Downloading Flowers102 dataset from datasets

In [21]:
#training_data = datasets.Flowers102(root="dataset", split="train", transform=train_transform, download=True)

val_data = datasets.Flowers102(root="dataset", split="val", transform=test_transform, download=True)

test_data = datasets.Flowers102(root="dataset", split="test", transform=test_transform, download=True)

In [22]:
batch_size = 64

#Create data loaders
#train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [23]:
#Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.model = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.BatchNorm2d(16),
      nn.Conv2d(16, 32, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.BatchNorm2d(32),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.BatchNorm2d(64),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.BatchNorm2d(128),
      nn.Flatten(),
      nn.Linear(128 * 14 * 14, 1024),
      nn.ReLU(),
      nn.Linear(1024, 512),
      nn.ReLU(),
      nn.Linear(512,256),
      nn.ReLU(),
      nn.Linear(256,102),
      nn.LogSoftmax(dim=1)
    )

  def forward(self, x):
    return self.model(x)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [25]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 5 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 200
for t in range(epochs):
  print(f"Epoch {t+1} \n ---------------------------------------")
  training_data = datasets.Flowers102(root="dataset", split="train", transform=train_transform, download=True)
  train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle=True)
  train(train_dataloader, model, loss_fn, optimizer)
  test(val_dataloader, model, loss_fn)
print("Testing -----------------------")
test(test_dataloader, model, loss_fn)
print("Done!!!!")

Epoch 1 
 ---------------------------------------
loss: 4.633262 [   64/ 1020]
loss: 4.575636 [  384/ 1020]
loss: 4.573678 [  704/ 1020]
loss: 4.266449 [  960/ 1020]
Test Error: 
 Accuracy: 2.2%, Avg loss: 4.581033 

Epoch 2 
 ---------------------------------------
loss: 4.329619 [   64/ 1020]
loss: 4.114266 [  384/ 1020]
loss: 3.928805 [  704/ 1020]
loss: 3.835431 [  960/ 1020]
Test Error: 
 Accuracy: 7.2%, Avg loss: 4.046039 

Epoch 3 
 ---------------------------------------
loss: 3.849914 [   64/ 1020]
loss: 4.009758 [  384/ 1020]
loss: 3.824996 [  704/ 1020]
loss: 3.527446 [  960/ 1020]
Test Error: 
 Accuracy: 12.6%, Avg loss: 3.749678 

Epoch 4 
 ---------------------------------------
loss: 3.629075 [   64/ 1020]
loss: 3.612241 [  384/ 1020]
loss: 3.464042 [  704/ 1020]
loss: 3.589995 [  960/ 1020]
Test Error: 
 Accuracy: 14.8%, Avg loss: 3.467619 

Epoch 5 
 ---------------------------------------
loss: 3.037247 [   64/ 1020]
loss: 3.509634 [  384/ 1020]
loss: 3.345223 [  704/